<a href="https://colab.research.google.com/github/GabrielMou21/projeto-bd-colab/blob/main/Projeto_BD_SQLite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
{
  "nbformat": 4,
  "nbformat_minor": 0,
  "metadata": {
    "colab": {
      "provenance": [],
      "authorship_tag": "ABX9TyPCn6ZHeh4hW00V8MIxYV1D",
      "include_colab_link": true
    },
    "kernelspec": {
      "name": "python3",
      "display_name": "Python 3"
    },
    "language_info": {
      "name": "python"
    }
  },
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {
        "id": "view-in-github",
        "colab_type": "text"
      },
      "source": [
        "<a href=\"https://colab.research.google.com/github/GabrielMou21/projeto-bd-colab/blob/main/Projeto_BD_SQLite.ipynb\" target=\"_parent\"><img src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/></a>"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": 1,
      "metadata": {
        "id": "HWZjqWirN2LU"
      },
      "outputs": [],
      "source": [
        "import sqlite3\n",
        "import pandas as pd"
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "conn = sqlite3.connect(':memory:')\n",
        "cursor = conn.cursor()\n",
        "cursor.execute(\"PRAGMA foreign_keys = ON;\")"
      ],
      "metadata": {
        "id": "qzjIIx3CPiTC"
      },
      "execution_count": 2,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute('''\n",
        "    CREATE TABLE IF NOT EXISTS clientes (\n",
        "        id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
        "        nome TEXT NOT NULL,\n",
        "        email TEXT UNIQUE,\n",
        "        data_cadastro DATE DEFAULT CURRENT_DATE\n",
        "    );\n",
        "''')"
      ],
      "metadata": {
        "id": "E5KQ1rTiPmIy"
      },
      "execution_count": 3,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute('''\n",
        "    CREATE TABLE IF NOT EXISTS produtos (\n",
        "        id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
        "        nome TEXT NOT NULL,\n",
        "        categoria TEXT,\n",
        "        preco REAL CHECK(preco > 0),\n",
        "        estoque INTEGER DEFAULT 0\n",
        "    );\n",
        "''')"
      ],
      "metadata": {
        "id": "A3Rfk-l9QVsS"
      },
      "execution_count": 4,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute('''\n",
        "    CREATE TABLE IF NOT EXISTS pedidos (\n",
        "        id INTEGER PRIMARY KEY AUTOINCREMENT,\n",
        "        cliente_id INTEGER,\n",
        "        data_pedido TIMESTAMP DEFAULT CURRENT_TIMESTAMP,\n",
        "        status TEXT DEFAULT 'Pendente',\n",
        "        FOREIGN KEY (cliente_id) REFERENCES clientes(id) ON DELETE CASCADE\n",
        "    );\n",
        "''')"
      ],
      "metadata": {
        "id": "q52jXCNoQa_i"
      },
      "execution_count": 5,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute('''\n",
        "    CREATE TABLE IF NOT EXISTS itens_pedido (\n",
        "        pedido_id INTEGER,\n",
        "        produto_id INTEGER,\n",
        "        quantidade INTEGER CHECK(quantidade > 0),\n",
        "        preco_unitario REAL,\n",
        "        PRIMARY KEY (pedido_id, produto_id),\n",
        "        FOREIGN KEY (pedido_id) REFERENCES pedidos(id) ON DELETE CASCADE,\n",
        "        FOREIGN KEY (produto_id) REFERENCES produtos(id)\n",
        "    );\n",
        "''')"
      ],
      "metadata": {
        "id": "WcYxKT1FQwMW"
      },
      "execution_count": 6,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.executemany(\"INSERT INTO clientes (nome, email) VALUES (?, ?)\",\n",
        "                  [('Ana Silva', 'ana@email.com'),\n",
        "                   ('Carlos Oliveira', 'carlos@email.com'),\n",
        "                   ('Mariana Souza', 'mariana@email.com'),\n",
        "                   ('João Santos', 'joao@email.com')])"
      ],
      "metadata": {
        "id": "_gyAUxd_QyG3"
      },
      "execution_count": 7,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute(\"SELECT * FROM clientes;\")\n",
        "results = cursor.fetchall()\n",
        "for row in results:\n",
        "    print(row)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/"
        },
        "id": "oQ9B-uwYRcWG",
        "outputId": "9f8f8e6a-9c7e-4c8f-9c2f-8c2c3b0c5d5a"
      },
      "execution_count": 8,
      "outputs": [
        {
          "output_type": "stream",
          "name": "stdout",
          "text": [
            "(1, 'Ana Silva', 'ana@email.com', None)\n",
            "(2, 'Carlos Oliveira', 'carlos@email.com', None)\n",
            "(3, 'Mariana Souza', 'mariana@email.com', None)\n",
            "(4, 'João Santos', 'joao@email.com', None)\n"
          ]
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.executemany(\"INSERT INTO produtos (nome, categoria, preco, estoque) VALUES (?, ?, ?, ?)\",\n",
        "                  [('Notebook Dell', 'Eletrônicos', 4500.00, 10),\n",
        "                   ('Smartphone Samsung', 'Eletrônicos', 2200.00, 15),\n",
        "                   ('Mesa de Escritório', 'Móveis', 800.00, 5),\n",
        "                   ('Cadeira Ergonômica', 'Móveis', 650.00, 8),\n",
        "                   ('Mouse Sem Fio', 'Acessórios', 120.00, 20)])"
      ],
      "metadata": {
        "id": "Txj6XVexRht8"
      },
      "execution_count": 9,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.executemany(\"INSERT INTO pedidos (cliente_id, status) VALUES (?, ?)\",\n",
        "                  [(1, 'Pendente'),\n",
        "                   (2, 'Processando'),\n",
        "                   (3, 'Pendente')])"
      ],
      "metadata": {
        "id": "E13WNyamRw8i"
      },
      "execution_count": 10,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.executemany(\"INSERT INTO itens_pedido VALUES (?, ?, ?, ?)\",\n",
        "                  [(1, 1, 1, 4500.00),\n",
        "                   (1, 5, 2, 120.00),\n",
        "                   (2, 2, 1, 2200.00),\n",
        "                   (3, 3, 1, 800.00),\n",
        "                   (3, 4, 2, 650.00)])"
      ],
      "metadata": {
        "id": "uVEJMtnGSO9m"
      },
      "execution_count": 11,
      "outputs": []
    },
    {
      "cell_type": "code",
      "source": [
        "cursor.execute('''\n",
        "    SELECT p.id, c.nome, pr.nome, ip.quantidade, ip.preco_unitario\n",
        "    FROM pedidos p\n",
        "    JOIN clientes c ON p.cliente_id = c.id\n",
        "    JOIN itens_pedido ip ON p.id = ip.pedido_id\n",
        "    JOIN produtos pr ON ip.produto_id = pr.id\n",
        "''')\n",
        "\n",
        "df = pd.DataFrame(cursor.fetchall(), \n",
        "                 columns=['Pedido', 'Cliente', 'Produto', 'Quantidade', 'Preço'])\n",
        "display(df)"
      ],
      "metadata": {
        "colab": {
          "base_uri": "https://localhost:8080/",
          "height": 175
        },
        "id": "3kM_VRTkSPTv",
        "outputId": "c5b6c3a3-1c8e-4f3e-8c5e-0c1e7a0c0a3e"
      },
      "execution_count": 12,
      "outputs": [
        {
          "output_type": "display_data",
          "data": {
            "text/plain": [
              "   Pedido          Cliente               Produto  Quantidade   Preço\n",
              "0       1        Ana Silva         Notebook Dell           1  4500.0\n",
              "1       1        Ana Silva          Mouse Sem Fio           2   120.0\n",
              "2       2  Carlos Oliveira  Smartphone Samsung           1  2200.0\n",
              "3       3    Mariana Souza    Mesa de Escritório           1   800.0\n",
              "4       3    Mariana Souza   Cadeira Ergonômica           2   650.0"
            ],
            "text/html": [
              "\n",
              "  <div id=\"df-8d5a9b0e-6a3f-4d7e-8c8a-9c9c5a3e5f5c\" class=\"colab-df-container\">\n",
              "    <div>\n",
              "<style scoped>\n",
              "    .dataframe tbody tr th:only-of-type {\n",
              "        vertical-align: middle;\n",
              "    }\n",
              "\n",
              "    .dataframe tbody tr th {\n",
              "        vertical-align: top;\n",
              "    }\n",
              "\n",
              "    .dataframe thead th {\n",
              "        text-align: right;\n",
              "    }\n",
              "</style>\n",
              "<table border=\"1\" class=\"dataframe\">\n",
              "  <thead>\n",
              "    <tr style=\"text-align: right;\">\n",
              "      <th></th>\n",
              "      <th>Pedido</th>\n",
              "      <th>Cliente</th>\n",
              "      <th>Produto</th>\n",
              "      <th>Quantidade</th>\n",
              "      <th>Preço</th>\n",
              "    </tr>\n",
              "  </thead>\n",
              "  <tbody>\n",
              "    <tr>\n",
              "      <th>0</th>\n",
              "      <td>1</td>\n",
              "      <td>Ana Silva</td>\n",
              "      <td>Notebook Dell</td>\n",
              "      <td>1</td>\n",
              "      <td>4500.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>1</th>\n",
              "      <td>1</td>\n",
              "      <td>Ana Silva</td>\n",
              "      <td>Mouse Sem Fio</td>\n",
              "      <td>2</td>\n",
              "      <td>120.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>2</th>\n",
              "      <td>2</td>\n",
              "      <td>Carlos Oliveira</td>\n",
              "      <td>Smartphone Samsung</td>\n",
              "      <td>1</td>\n",
              "      <td>2200.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>3</th>\n",
              "      <td>3</td>\n",
              "      <td>Mariana Souza</td>\n",
              "      <td>Mesa de Escritório</td>\n",
              "      <td>1</td>\n",
              "      <td>800.0</td>\n",
              "    </tr>\n",
              "    <tr>\n",
              "      <th>4</th>\n",
              "      <td>3</td>\n",
              "      <td>Mariana Souza</td>\n",
              "      <td>Cadeira Ergonômica</td>\n",
              "      <td>2</td>\n",
              "      <td>650.0</td>\n",
              "    </tr>\n",
              "  </tbody>\n",
              "</table>\n",
              "</div>\n",
              "    <div class=\"colab-df-buttons\">\n",
              "\n",
              "  <div class=\"colab-df-container\">\n",
              "    <button class=\"colab-df-convert\" onclick=\"convertToInteractive('df-8d5a9b0e-6a3f-4d7e-8c8a-9c9c5a3e5f5c')\"\n",
              "            title=\"Convert this dataframe to an interactive table.\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "  <svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\" viewBox=\"0 -960 960 960\">\n",
              "    <path d=\"M120-120v-720h720v720H120Zm60-500h600v-160H180v160Zm220 220h160v-160H400v160Zm0 220h160v-160H400v160ZM180-400h160v-160H180v160Zm440 0h160v-160H620v160ZM180-180h160v-160H180v160Zm440 0h160v-160H620v160Z\"/>\n",
              "  </svg>\n",
              "    </button>\n",
              "\n",
              "  <style>\n",
              "    .colab-df-container {\n",
              "      display:flex;\n",
              "      gap: 12px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert {\n",
              "      background-color: #E8F0FE;\n",
              "      border: none;\n",
              "      border-radius: 50%;\n",
              "      cursor: pointer;\n",
              "      display: none;\n",
              "      fill: #1967D2;\n",
              "      height: 32px;\n",
              "      padding: 0 0 0 0;\n",
              "      width: 32px;\n",
              "    }\n",
              "\n",
              "    .colab-df-convert:hover {\n",
              "      background-color: #E2EBFA;\n",
              "      box-shadow: 0px 1px 2px rgba(60, 64, 67, 0.3), 0px 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "      fill: #174EA6;\n",
              "    }\n",
              "\n",
              "    .colab-df-buttons div {\n",
              "      margin-bottom: 4px;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert {\n",
              "      background-color: #3B4455;\n",
              "      fill: #D2E3FC;\n",
              "    }\n",
              "\n",
              "    [theme=dark] .colab-df-convert:hover {\n",
              "      background-color: #434B5C;\n",
              "      box-shadow: 0px 1px 3px 1px rgba(0, 0, 0, 0.15);\n",
              "      filter: drop-shadow(0px 1px 2px rgba(0, 0, 0, 0.3));\n",
              "      fill: #FFFFFF;\n",
              "    }\n",
              "  </style>\n",
              "\n",
              "    <script>\n",
              "      const buttonEl =\n",
              "        document.querySelector('#df-8d5a9b0e-6a3f-4d7e-8c8a-9c9c5a3e5f5c button.colab-df-convert');\n",
              "      buttonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "\n",
              "      async function convertToInteractive(key) {\n",
              "        const element = document.querySelector('#df-8d5a9b0e-6a3f-4d7e-8c8a-9c9c5a3e5f5c');\n",
              "        const dataTable =\n",
              "          await google.colab.kernel.invokeFunction('convertToInteractive',\n",
              "                                                    [key], {});\n",
              "        if (!dataTable) return;\n",
              "\n",
              "        const docLinkHtml = 'Like what you see? Visit the ' +\n",
              "          '<a target=\"_blank\" href=https://colab.research.google.com/notebooks/data_table.ipynb>data table notebook</a>'\n",
              "          + ' to learn more about interactive tables.';\n",
              "        element.innerHTML = '';\n",
              "        dataTable['output_type'] = 'display_data';\n",
              "        await google.colab.output.renderOutput(dataTable, element);\n",
              "        const docLink = document.createElement('div');\n",
              "        docLink.innerHTML = docLinkHtml;\n",
              "        element.appendChild(docLink);\n",
              "      }\n",
              "    </script>\n",
              "  </div>\n",
              "\n",
              "\n",
              "<div id=\"df-4a9a7f8e-1e6b-4f6f-8b7f-3b2a3c1c0c4e\">\n",
              "  <button class=\"colab-df-quickchart\" onclick=\"quickchart('df-4a9a7f8e-1e6b-4f6f-8b7f-3b2a3c1c0c4e')\"\n",
              "            title=\"Suggest charts\"\n",
              "            style=\"display:none;\">\n",
              "\n",
              "<svg xmlns=\"http://www.w3.org/2000/svg\" height=\"24px\"viewBox=\"0 0 24 24\"\n",
              "     width=\"24px\">\n",
              "    <g>\n",
              "        <path d=\"M19 3H5c-1.1 0-2 .9-2 2v14c0 1.1.9 2 2 2h14c1.1 0 2-.9 2-2V5c0-1.1-.9-2-2-2zM9 17H7v-7h2v7zm4 0h-2V7h2v10zm4 0h-2v-4h2v4z\"/>\n",
              "    </g>\n",
              "</svg>\n",
              "  </button>\n",
              "\n",
              "<style>\n",
              "  .colab-df-quickchart {\n",
              "      --bg-color: #E8F0FE;\n",
              "      --fill-color: #1967D2;\n",
              "      --hover-bg-color: #E2EBFA;\n",
              "      --hover-fill-color: #174EA6;\n",
              "      --disabled-fill-color: #AAA;\n",
              "      --disabled-bg-color: #DDD;\n",
              "  }\n",
              "\n",
              "  [theme=dark] .colab-df-quickchart {\n",
              "      --bg-color: #3B4455;\n",
              "      --fill-color: #D2E3FC;\n",
              "      --hover-bg-color: #434B5C;\n",
              "      --hover-fill-color: #FFFFFF;\n",
              "      --disabled-fill-color: #666;\n",
              "      --disabled-bg-color: #555;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart {\n",
              "    background-color: var(--bg-color);\n",
              "    border: none;\n",
              "    border-radius: 50%;\n",
              "    cursor: pointer;\n",
              "    display: none;\n",
              "    fill: var(--fill-color);\n",
              "    height: 32px;\n",
              "    padding: 0;\n",
              "    width: 32px;\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart:hover {\n",
              "    background-color: var(--hover-bg-color);\n",
              "    box-shadow: 0 1px 2px rgba(60, 64, 67, 0.3), 0 1px 3px 1px rgba(60, 64, 67, 0.15);\n",
              "    fill: var(--button-hover-fill-color);\n",
              "  }\n",
              "\n",
              "  .colab-df-quickchart-complete:disabled,\n",
              "  .colab-df-quickchart-complete:disabled:hover {\n",
              "    background-color: var(--disabled-bg-color);\n",
              "    fill: var(--disabled-fill-color);\n",
              "    box-shadow: none;\n",
              "  }\n",
              "\n",
              "  .colab-df-spinner {\n",
              "    border: 2px solid var(--fill-color);\n",
              "    border-color: transparent;\n",
              "    border-bottom-color: var(--fill-color);\n",
              "    animation:\n",
              "      spin 1s steps(1) infinite;\n",
              "  }\n",
              "\n",
              "  @keyframes spin {\n",
              "    0% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "      border-left-color: var(--fill-color);\n",
              "    }\n",
              "    20% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    30% {\n",
              "      border-color: transparent;\n",
              "      border-left-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    40% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-top-color: var(--fill-color);\n",
              "    }\n",
              "    60% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "    }\n",
              "    80% {\n",
              "      border-color: transparent;\n",
              "      border-right-color: var(--fill-color);\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "    90% {\n",
              "      border-color: transparent;\n",
              "      border-bottom-color: var(--fill-color);\n",
              "    }\n",
              "  }\n",
              "</style>\n",
              "\n",
              "  <script>\n",
              "    async function quickchart(key) {\n",
              "      const quickchartButtonEl =\n",
              "        document.querySelector('#' + key + ' button');\n",
              "      quickchartButtonEl.disabled = true;  // To prevent multiple clicks.\n",
              "      quickchartButtonEl.classList.add('colab-df-spinner');\n",
              "      try {\n",
              "        const charts = await google.colab.kernel.invokeFunction(\n",
              "            'suggestCharts', [key], {});\n",
              "      } catch (error) {\n",
              "        console.error('Error during call to suggestCharts:', error);\n",
              "      }\n",
              "      quickchartButtonEl.classList.remove('colab-df-spinner');\n",
              "      quickchartButtonEl.classList.add('colab-df-quickchart-complete');\n",
              "    }\n",
              "    (() => {\n",
              "      let quickchartButtonEl =\n",
              "        document.querySelector('#df-4a9a7f8e-1e6b-4f6f-8b7f-3b2a3c1c0c4e button');\n",
              "      quickchartButtonEl.style.display =\n",
              "        google.colab.kernel.accessAllowed ? 'block' : 'none';\n",
              "    })();\n",
              "  </script>\n",
              "</div>\n",
              "\n",
              "    </div>\n",
              "  </div>\n"
            ],
            "application/vnd.google.colaboratory.intrinsic+json": {
              "type": "dataframe",
              "variable_name": "df",
              "summary": "{\n  \"name\": \"df\",\n  \"rows\": 5,\n  \"fields\": [\n    {\n      \"column\": \"Pedido\",\n      \"properties\": {\n        \"dtype\": \"number\",\n        \"std\": 0,\n        \"min\": 1,\n        \"max\": 3,\n        \"num_unique_values\": 3,\n        \"samples\": [\n          1,\n          2,\n          3\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"Cliente\",\n      \"properties\": {\n        \"dtype\": \"string\",\n        \"num_unique_values\": 3,\n        \"samples\": [\n          \"Ana Silva\",\n          \"Carlos Oliveira\",\n          \"Mariana Souza\"\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"Produto\",\n      \"properties\": {\n        \"dtype\": \"string\",\n        \"num_unique_values\": 5,\n        \"samples\": [\n          \"Mouse Sem Fio\",\n          \"Smartphone Samsung\",\n          \"Cadeira Ergon\\u00f4mica\"\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"Quantidade\",\n      \"properties\": {\n        \"dtype\": \"number\",\n        \"std\": 0,\n        \"min\": 1,\n        \"max\": 2,\n        \"num_unique_values\": 2,\n        \"samples\": [\n          2,\n          1\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    },\n    {\n      \"column\": \"Pre\\u00e7o\",\n      \"properties\": {\n        \"dtype\": \"number\",\n        \"std\": 1735.3393548633734,\n        \"min\": 120.0,\n        \"max\": 4500.0,\n        \"num_unique_values\": 5,\n        \"samples\": [\n          120.0,\n          2200.0\n        ],\n        \"semantic_type\": \"\",\n        \"description\": \"\"\n      }\n    }\n  ]\n}"
            }
          },
          "metadata": {},
          "execution_count": 12
        }
      ]
    },
    {
      "cell_type": "code",
      "source": [
        "conn.close()"
      ],
      "metadata": {
        "id": "3NizALImSPTv"
      },
      "execution_count": 13,
      "outputs": []
    }
  ]
}